In [13]:
import pandas as pd
import requests

import openmeteo_requests
import requests_cache
from retry_requests import retry

### Update Polen

In [17]:
URL_POLEN = "https://datos.comunidad.madrid/catalogo/dataset/e608aace-3593-43a3-8c91-02332137fa83/resource/db5e3952-57f5-40f3-bb1d-906eb17aebb1/download/mediciones_polen.json"
CSV_POLEN = "new_datasets\gramineas_2014_2025.csv"

response = requests.get(URL_POLEN)
nuevos_datos = response.json()
df_nuevos = pd.DataFrame(nuevos_datos['data'] if 'data' in nuevos_datos else nuevos_datos)

df_nuevos = df_nuevos[df_nuevos['tipo_polinico'] == 'Gramíneas']
df_nuevos = df_nuevos[df_nuevos['captador'] == 'AYTM']
df_nuevos['fecha'] = pd.to_datetime(df_nuevos['fecha_lectura']).dt.date
df_nuevos.drop(columns=['fecha_lectura','captador'], inplace=True)

df_existente = pd.read_csv(CSV_POLEN)
df_final = pd.concat([df_existente, df_nuevos]).drop_duplicates().reset_index(drop=True)
df_final = df_final.groupby('fecha', as_index=False).first()

df_final.to_csv('new_datasets/gramineas_2014_2025.csv', index=False, encoding='utf-8')

<>:2: SyntaxWarning: invalid escape sequence '\g'
<>:2: SyntaxWarning: invalid escape sequence '\g'
C:\Users\sergi\AppData\Local\Temp\ipykernel_13464\3774609062.py:2: SyntaxWarning: invalid escape sequence '\g'
  CSV_POLEN = "new_datasets\gramineas_2014_2025.csv"


### Update Meteo

In [ ]:
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 40.4165,
	"longitude": -3.7026,
	"hourly": ["temperature_2m", "wind_speed_10m", "wind_gusts_10m", "relative_humidity_2m", "wind_direction_10m", "dew_point_2m", "rain", "vapour_pressure_deficit", "et0_fao_evapotranspiration", "cloud_cover", "shortwave_radiation"],
	"past_days": 31,
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_gusts_10m = hourly.Variables(2).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(3).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(4).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(5).ValuesAsNumpy()
hourly_rain = hourly.Variables(6).ValuesAsNumpy()
hourly_vapour_pressure_deficit = hourly.Variables(7).ValuesAsNumpy()
hourly_et0_fao_evapotranspiration = hourly.Variables(8).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(9).ValuesAsNumpy()
hourly_shortwave_radiation = hourly.Variables(10).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m (°C)"] = hourly_temperature_2m
hourly_data["wind_speed_10m (km/h)"] = hourly_wind_speed_10m
hourly_data["wind_gusts_10m (km/h)"] = hourly_wind_gusts_10m
hourly_data["relative_humidity_2m (%)"] = hourly_relative_humidity_2m
hourly_data["wind_direction_10m (°)"] = hourly_wind_direction_10m
hourly_data["dew_point_2m (°C)"] = hourly_dew_point_2m
hourly_data["rain (mm)"] = hourly_rain
hourly_data["vapour_pressure_deficit (kPa)"] = hourly_vapour_pressure_deficit
hourly_data["et0_fao_evapotranspiration (mm)"] = hourly_et0_fao_evapotranspiration
hourly_data["cloud_cover (%)"] = hourly_cloud_cover
hourly_data["shortwave_radiation (W/m²)"] = hourly_shortwave_radiation

hourly_dataframe = pd.DataFrame(data = hourly_data)

hourly_dataframe['fecha'] = pd.to_datetime(hourly_dataframe['date']).dt.date
df_meteo = hourly_dataframe.groupby('fecha').mean(numeric_only=True).reset_index()

df_final = pd.concat([df_existente, df_meteo], ignore_index=True)
df_final = df_final.groupby('fecha', as_index=False).first()
df_final.to_csv('new_datasets/gramineas_2014_2025.csv', index=False, encoding='utf-8')

Coordinates: 40.4375°N -3.6875°E
Elevation: 651.0 m asl
Timezone difference to GMT+0: 0s
